In [2]:
# Imports
import pandas as pd

In [291]:
# This is where we load all the data through oracle; but for now I'll just read.csv the data in.

emissions = pd.read_csv("data/tidied_emissions_with_key.csv")
employment = pd.read_csv("data/tidied_employment_estimates.csv")
industry = pd.read_csv("data/tidied_industry_mapping.csv")
population = pd.read_csv("data/tidied_population_estimates.csv")

In [292]:
## model1: emissions/employed (by year, province) ~ good type(dummy) + province (dummy) + year

# joining emissions and industry together
emissions_industry = pd.merge(emissions, industry, left_on="industry_code", right_on="IndustryCode")

# filter out canada in emissions_industry dataset
emissions_industry = emissions_industry[emissions_industry["province"] != "Canada"]

# mutate a new column called ProvinceYearDurable in EmissionsIndustry
emissions_industry["ProvinceYearDurable"] = emissions_industry["province"] + emissions_industry["year"].astype(str) + emissions_industry["Durable"].astype(str)

# remove households, put in separate dataframe
emissions_household = emissions_industry[emissions_industry["ioic_code"].isin(['9999999', '9999998'])]
emissions_industry = emissions_industry[~emissions_industry["ioic_code"].isin(['9999999', '9999998'])]

# drop a few duplicate columns
emissions_industry.drop(columns=["IndustryCode", "IndustryName", "IOIC", "industry_name", "industry_code", "NAICS"], inplace=True)

# rename count to "emissions"
emissions_industry.rename(columns={'count': 'emissions'}, inplace=True)

# group by Province, Year, Durable, sum emissions for each industry
grouped_emissions = pd.DataFrame(emissions_industry.groupby(['province', 'year', 'Durable'])['emissions'].sum())
grouped_emissions = grouped_emissions.reset_index()
grouped_emissions["ProvinceYearDurable"] = grouped_emissions["province"] + grouped_emissions["year"].astype(str) + grouped_emissions["Durable"].astype(str)




# remove Manufacturing, Canada from naics_industry
employment_tidy = employment[(employment["naics_industry"] != "Manufacturing") & (employment["province"] != "Canada")]

# rename employment estimate data's manufacturing column to 0 or 1
employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
employment_tidy = employment_tidy.drop(columns=['naics_industry'])

# mutate a new column called ProvinceYearDurable in employment
employment_tidy["ProvinceYearDurable"] = employment_tidy["province"] + employment_tidy["year"].astype(str) + employment_tidy["durable"].astype(str)

# remove Manufacturing, Canada from naics_industry
employment_tidy = employment[(employment["naics_industry"] != "Manufacturing") & (employment["province"] != "Canada")]

# rename employment estimate data's manufacturing column to 0 or 1
employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
employment_tidy = employment_tidy.drop(columns=['naics_industry'])

# mutate a new column called ProvinceYearDurable in employment
employment_tidy["ProvinceYearDurable"] = employment_tidy["province"] + employment_tidy["year"].astype(str) + employment_tidy["durable"].astype(str)
employment_tidy["employed"] = employment_tidy["count"]
employment_tidy = employment_tidy.drop(columns=["count", "province", "year", "durable"])




# join the grouped_emissions with employment_tidy, drop key
emissions_industry_employment = pd.merge(grouped_emissions, employment_tidy, on="ProvinceYearDurable")
emissions_industry_employment = emissions_industry_employment.drop(columns="ProvinceYearDurable")


/tmp/ipykernel_156/3112970236.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
/tmp/ipykernel_156/3112970236.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})


In [269]:
emissions_industry_employment

,province,year,Durable,emissions,employed
0,Alberta,2012,0,27513,50.6
1,Alberta,2012,1,4249,88.8
2,Alberta,2013,0,28619,59.9
3,Alberta,2013,1,4921,82.8
4,Alberta,2014,0,27933,56.5
...,...,...,...,...,...
195,Saskatchewan,2019,1,775,18.4
196,Saskatchewan,2020,0,3011,11.9
197,Saskatchewan,2020,1,762,18.8
198,Saskatchewan,2021,0,3135,12.9


In [311]:
## model2: emissions/population (by year, province) ~ province(dummy) * year ---> households
emissions_household_tidy = emissions_household.drop(columns=["ioic_code", "IndustryCode", "IndustryName", "industry_name", "IOIC", "Durable"])

# rename count to "emissions"
emissions_household_tidy.rename(columns={'count': 'emissions'}, inplace=True)

# group by Province, Year, Durable, sum emissions for each household use
grouped_emissions_hh = pd.DataFrame(emissions_household_tidy.groupby(['province', 'year'])['emissions'].sum())
grouped_emissions_hh = grouped_emissions_hh.reset_index()
grouped_emissions_hh["ProvinceYear"] = grouped_emissions_hh["province"] + grouped_emissions_hh["year"].astype(str)



# Remove Canada, rename column from Geography to Province
population_tidy = population[population["Geography"] != "Canada"]
population_tidy.rename(columns={'Geography': 'Province'}, inplace=True)

# Make a Province Year key, drop province, drop year
population_tidy["ProvinceYear"] = population_tidy["Province"] + population_tidy["year"].astype(str)
population_tidy.drop(columns=["Province", "year"], inplace=True)

# join the grouped_emissions_hh with population_tidy, drop key, rename count to Population
emissions_household_population = pd.merge(grouped_emissions_hh, population_tidy, on="ProvinceYear")
emissions_household_population.drop(columns="ProvinceYear", inplace=True)
emissions_household_population.rename(columns={'count': 'population'}, inplace=True)

# create response variable: emissions/population
emissions_household_population["emissions_per_capita"] = emissions_household_population["emissions"]/emissions_household_population["population"]
emissions_household_population.drop(columns=["emissions", "population"], inplace=True)

emissions_household_population

/tmp/ipykernel_156/1664319845.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_tidy.rename(columns={'Geography': 'Province'}, inplace=True)
/tmp/ipykernel_156/1664319845.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_tidy["ProvinceYear"] = population_tidy["Province"] + population_tidy["year"].astype(str)
/tmp/ipykernel_156/1664319845.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

,province,year,emissions_per_capita
0,Alberta,2012,0.004781
1,Alberta,2013,0.004969
2,Alberta,2014,0.004884
3,Alberta,2015,0.004505
4,Alberta,2016,0.004520
...,...,...,...
125,Yukon,2017,0.003501
126,Yukon,2018,0.003627
127,Yukon,2019,0.003859
128,Yukon,2020,0.003325


# linear regression: 

models:

1. emissions/population (by year, province) ~ province(dummy) * year ---> households
2. emissions/employed (by year, province) ~ good type(dummy) + province (dummy) + year